In [ ]:
import os
import cv2
import pandas as pd
import pytesseract
from ultralytics import YOLO
import numpy as np
import time

# -----------------------------------------------------------------
# Cella 1: Configurazione
# -----------------------------------------------------------------
print("Configurazione del progetto...")

# --- IMPOSTAZIONI PRINCIPALI ---

# Percorso del modello YOLO allenato
MODEL_PATH = r"C:\Users\ricca\OneDrive - Alma Mater Studiorum Università di Bologna\Desktop\Vision Por computador\lab\4\yolo\yolo\my_model\train\weights\best.pt"

# File video da analizzare
#VIDEO_SOURCE_PATH = r"C:\Users\ricca\OneDrive - Alma Mater Studiorum Università di Bologna\Desktop\Vision Por computador\lab\4\yolo\yolo\videoProva.mp4"
VIDEO_SOURCE_PATH = r"C:\Users\ricca\OneDrive - Alma Mater Studiorum Università di Bologna\Desktop\Vision Por computador\lab\4\video\videoTelefono4.mp4"

# Nome del file video di output
OUTPUT_VIDEO_PATH = "risultato_Tesseract2.mp4"

# Nome del file CSV di output (formato richiesto)
OUTPUT_CSV_PATH = "risultati_Tesseract2.csv"

# Soglia di confidenza minima
CONF_THRESHOLD = 0.4 

# -----------------------------------------------------------------
# Cella 2: Importazioni e Caricamento Modelli
# -----------------------------------------------------------------
print("Caricamento modelli...")

# Configura il percorso di Tesseract
try:
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    pytesseract.get_tesseract_version()
    print("Tesseract trovato e configurato.")
except Exception as e:
    print(f"ERRORE: Tesseract non trovato. Installalo o correggi il percorso: {e}")

# Carica il modello YOLOv8 per il rilevamento e tracking
try:
    model = YOLO(MODEL_PATH)
    print("Scaldando il modello YOLO...")
    dummy_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
    model.track(dummy_frame, persist=True, verbose=False)
    print("Modello YOLO caricato e pronto.")
except Exception as e:
    print(f"Errore nel caricamento del modello YOLO: {e}")
    # raise e 

# -----------------------------------------------------------------
# Cella 3: Funzioni di Supporto (OCR e Pre-processing)
# -----------------------------------------------------------------
print("Definizione funzioni di supporto...")

def canonical_classname(name: str) -> str:
    """Normalizza i nomi delle classi."""
    s = name.strip().lower().replace('_', ' ').replace('-', ' ')
    plate_aliases = {'license plate','licence plate','number plate','plate','vehicle plate','car plate','licence','license', 'matricula'}
    moto_aliases = {'motorcycle','motorbike','moped','scooter', 'moto'}
    car_aliases = {'car','automobile','vehicle','auto', 'coche'}
    person_aliases = {'person','pedestrian','people', 'persona'}
    if s in plate_aliases: return 'license-plate'
    if s in moto_aliases: return 'motorcycle'
    if s in car_aliases: return 'car'
    if s in person_aliases: return 'person'
    return s

def get_box_center(box):
    """Calcola il centro di un bounding box."""
    x1, y1, x2, y2 = box
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

def run_ocr_on_plate(plate_image):
    """
    Esegue Tesseract su un'immagine di targa pre-processata.
    Questa funzione ora restituisce solo il testo pulito, nient'altro.
    """
    if plate_image.size == 0:
        return ""

    text = ""
    try:
        # --- Pre-processing AVANZATO per Tesseract ---
        h, w = plate_image.shape[:2]
        if h == 0 or w == 0: return ""
        
        scale_factor = 4
        img_large = cv2.resize(plate_image, (w * scale_factor, h * scale_factor), interpolation=cv2.INTER_CUBIC)
        gray = cv2.cvtColor(img_large, cv2.COLOR_BGR2GRAY)
        gray_blurred = cv2.medianBlur(gray, 3)
        _, thresh = cv2.threshold(gray_blurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        thresh_border = cv2.copyMakeBorder(thresh, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[255,255,255])
        
        # --- Configurazione Tesseract ---
        custom_config = r'--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        
        text = pytesseract.image_to_string(thresh_border, config=custom_config)
        
        # Pulisci il testo da spazi e caratteri speciali
        text = "".join(c for c in text if c.isalnum())

    except Exception as e:
        return ""
    
    return text # Ritorna solo il testo pulito


# -----------------------------------------------------------------
# Cella 4: Elaborazione Principale
# -----------------------------------------------------------------
print("Inizio elaborazione video...")

csv_results = []
start_time = time.time()

# --- MODIFICA: Dizionari per la "Memoria" OCR ---
# Salva la storia delle letture per ogni track_id
ocr_history = {} 
# Salva il testo stabile e la confidenza per ogni track_id
stable_ocr_results = {} 

# Set per tenere traccia degli ID unici per il conteggio totale
unique_object_ids = {
    'car': set(), 'person': set(), 'license-plate': set(), 'motorcycle': set()
}

cap = cv2.VideoCapture(VIDEO_SOURCE_PATH)
if not cap.isOpened():
    print(f"ERRORE: Impossibile aprire il video sorgente: {VIDEO_SOURCE_PATH}")
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    recorder = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("\nElaborazione video completata.")
            break

        frame_number += 1
        
        # Esegui il tracking con YOLO
        results = model.track(frame, persist=True, conf=CONF_THRESHOLD, verbose=False)

        # Set per tenere traccia degli ID attivi in questo frame
        current_frame_active_ids = set()

        if results[0].boxes and len(results[0].boxes) > 0:
            boxes = results[0].boxes.cpu().numpy()
            class_names = results[0].names
            
            cars_in_frame = []
            plates_in_frame = []
            people_in_frame = []
            motos_in_frame = []

            has_tracking_ids = boxes.id is not None

            for i in range(len(boxes)):
                class_id = int(boxes.cls[i])
                class_name = canonical_classname(class_names[class_id])
                track_id = int(boxes.id[i]) if has_tracking_ids else -1
                conf = float(boxes.conf[i])
                bbox = boxes.xyxy[i].astype(int)
                
                obj_data = {'bbox': bbox, 'track_id': track_id, 'conf': conf, 'class_name': class_name}

                if track_id != -1:
                    # Aggiungi l'ID al set per il conteggio unico
                    if class_name in unique_object_ids:
                        unique_object_ids[class_name].add(track_id)
                    # Aggiungi agli ID attivi per la pulizia della memoria OCR
                    if class_name == 'license-plate':
                        current_frame_active_ids.add(track_id)

                if class_name == 'car':
                    cars_in_frame.append(obj_data)
                elif class_name == 'license-plate':
                    plates_in_frame.append(obj_data)
                elif class_name == 'person':
                    people_in_frame.append(obj_data)
                elif class_name == 'motorcycle':
                    motos_in_frame.append(obj_data)

            # --- MODIFICA: Logica OCR con Memoria ---
            for plate in plates_in_frame:
                track_id = plate['track_id']
                if track_id == -1: # Salta se il tracker non ha ancora assegnato un ID
                    continue

                x1, y1, x2, y2 = plate['bbox']
                plate_img = frame[y1:y2, x1:x2]
                
                # Esegui l'OCR (potresti limitarlo, es. "if frame_number % 5 == 0")
                text = run_ocr_on_plate(plate_img) # Usa la funzione Tesseract
                
                # Se Tesseract ha letto qualcosa, aggiorna la memoria
                if text:
                    if track_id not in ocr_history:
                        ocr_history[track_id] = {} # Inizializza la memoria per questo ID
                    
                    # Aggiungi un "voto" per questo testo
                    ocr_history[track_id][text] = ocr_history[track_id].get(text, 0) + 1
                
                # Ora, determina il testo STABILE e la sua confidenza
                if track_id in ocr_history:
                    history_for_id = ocr_history[track_id]
                    
                    # Trova il testo con più voti
                    best_text = max(history_for_id, key=history_for_id.get)
                    
                    # Calcola la "confidenza di stabilità"
                    total_votes = sum(history_for_id.values())
                    best_votes = history_for_id[best_text]
                    stability_conf = best_votes / total_votes
                    
                    # Salva il risultato stabile
                    plate['text'] = best_text
                    plate['text_conf'] = stability_conf
                
                # Disegna il riquadro della targa
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                
                # Disegna il testo STABILE e la sua confidenza
                if plate.get('text', ''): 
                    text_to_draw = f"{plate['text']} ({plate.get('text_conf', 0):.2f})"
                    cv2.putText(frame, text_to_draw, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # --- Associa targhe alle auto e popola il CSV ---
            # (Questa logica ora usa automaticamente il testo stabile)
            for car in cars_in_frame:
                car_bbox = car['bbox']
                associated_plate = None
                for plate in plates_in_frame:
                    center_x, center_y = get_box_center(plate['bbox'])
                    if car_bbox[0] < center_x < car_bbox[2] and car_bbox[1] < center_y < car_bbox[3]:
                        associated_plate = plate
                        break
                
                row = {
                    'fotograma': frame_number, 'tipo_oggetto': 'car', 'confianza': f"{car['conf']:.2f}",
                    'identificador_tracking': f"car_{car['track_id']}", 'x1': car_bbox[0], 'y1': car_bbox[1], 'x2': car_bbox[2], 'y2': car_bbox[3],
                    'matrícula_en_su_caso': 'no', 'confianza_matricula': 0, 'mx1': 0, 'my1': 0, 'mx2': 0, 'my2': 0, 'texto_matricula': ''
                }

                if associated_plate:
                    plate_bbox = associated_plate['bbox']
                    row.update({
                        'matrícula_en_su_caso': 'si',
                        'confianza_matricula': f"{associated_plate.get('text_conf', 0):.2f}", # Usa la confidenza di stabilità
                        'mx1': plate_bbox[0], 'my1': plate_bbox[1], 'mx2': plate_bbox[2], 'my2': plate_bbox[3],
                        'texto_matricula': associated_plate.get('text', '') # Usa il testo stabile
                    })
                
                csv_results.append(row)
                
                cv2.rectangle(frame, (car_bbox[0], car_bbox[1]), (car_bbox[2], car_bbox[3]), (0, 255, 0), 2)
                cv2.putText(frame, f"Car ID:{car['track_id']}", (car_bbox[0], car_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Aggiungi le persone al CSV e disegnale
            for person in people_in_frame:
                p_bbox = person['bbox']
                csv_results.append({
                    'fotograma': frame_number, 'tipo_oggetto': 'person', 'confianza': f"{person['conf']:.2f}",
                    'identificador_tracking': f"person_{person['track_id']}", 'x1': p_bbox[0], 'y1': p_bbox[1], 'x2': p_bbox[2], 'y2': p_bbox[3],
                    'matrícula_en_su_caso': 'no', 'confianza_matricula': 0, 'mx1': 0, 'my1': 0, 'mx2': 0, 'my2': 0, 'texto_matricula': ''
                })
                cv2.rectangle(frame, (p_bbox[0], p_bbox[1]), (p_bbox[2], p_bbox[3]), (255, 0, 0), 2)
                cv2.putText(frame, f"Person ID:{person['track_id']}", (p_bbox[0], p_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Aggiungi le moto al CSV e disegnale
            for moto in motos_in_frame:
                m_bbox = moto['bbox']
                csv_results.append({
                    'fotograma': frame_number, 'tipo_oggetto': 'motorcycle', 'confianza': f"{moto['conf']:.2f}",
                    'identificador_tracking': f"motorcycle_{moto['track_id']}", 'x1': m_bbox[0], 'y1': m_bbox[1], 'x2': m_bbox[2], 'y2': m_bbox[3],
                    'matrícula_en_su_caso': 'no', 'confianza_matricula': 0, 'mx1': 0, 'my1': 0, 'mx2': 0, 'my2': 0, 'texto_matricula': ''
                })
                cv2.rectangle(frame, (m_bbox[0], m_bbox[1]), (m_bbox[2], m_bbox[3]), (0, 165, 255), 2) # Colore arancione
                cv2.putText(frame, f"Moto ID:{moto['track_id']}", (m_bbox[0], m_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        recorder.write(frame)
        
        if frame_number % 100 == 0:
            print(f"  Elaborato frame {frame_number}...")

        # Mostra un'anteprima
        cv2.imshow("Anteprima - Tesseract STABILE", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        # --- MODIFICA: Pulizia della memoria OCR ---
        # Rimuovi gli ID che non sono più attivi
        for old_id in list(ocr_history.keys()):
            if old_id not in current_frame_active_ids:
                del ocr_history[old_id]

    # Rilascia le risorse
    cap.release()
    recorder.release()
    cv2.destroyAllWindows()
    for i in range(5): cv2.waitKey(1) # Forza la chiusura delle finestre
    
    end_time = time.time()
    print(f"Tempo totale di elaborazione: {end_time - start_time:.2f} secondi.")

# -----------------------------------------------------------------
# Celda 5: Guardado de CSV y Conteo Total
# -----------------------------------------------------------------
print("Guardando archivo CSV...")

if csv_results:
    df = pd.DataFrame(csv_results)
    
    # Asegurar el orden correcto de las columnas según lo requerido
    colonne_richieste = [
        'fotograma', 'tipo_oggetto', 'confianza', 'identificador_tracking',
        'x1', 'y1', 'x2', 'y2', 'matrícula_en_su_caso', 'confianza_matricula',
        'mx1', 'my1', 'mx2', 'my2', 'texto_matricula'
    ]
    
    # Rellenar columnas faltantes si no existen (seguridad)
    for col in colonne_richieste:
        if col not in df.columns:
            # Crear una columna vacía con el tipo de dato correcto
            if 'x' in col or 'y' in col or 'fotograma' in col:
                df[col] = 0
            elif 'texto' in col:
                df[col] = ''
            elif 'confianza' in col:
                df[col] = 0.0
            else:
                df[col] = 'no'
            
    df = df[colonne_richieste]
    
    df.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"¡Guardado completado! Video guardado en '{OUTPUT_VIDEO_PATH}' y CSV en '{OUTPUT_CSV_PATH}'.")
else:
    print("No se detectaron objetos durante el procesamiento. No se guardó ningún archivo CSV.")

# MODIFICACIÓN: Imprimir y guardar el conteo total y el tiempo
print("\n--- Resumen de Conteo Único (cuente el total de cada clase) ---")

# Calcular totales
total_cars = len(unique_object_ids.get('car', set()))
total_people = len(unique_object_ids.get('person', set()))
total_plates = len(unique_object_ids.get('license-plate', set()))
total_motos = len(unique_object_ids.get('motorcycle', set()))

# Calcular tiempo (usando las variables 'end_time' y 'start_time' de la Celda 4)
try:
    processing_time = end_time - start_time
except NameError:
    print("Advertencia: No se encontraron las variables start_time/end_time. El tiempo no se guardará.")
    processing_time = -1.0 # Valor por defecto

# 1. Imprimir en consola (incluyendo el tiempo)
print(f"Total de coches únicos rastreados: {total_cars}")
print(f"Total de personas únicas rastreadas: {total_people}")
print(f"Total de motos únicas rastreadas: {total_motos}")
print(f"Total de matrículas únicas rastreadas: {total_plates}")
print(f"\nTiempo total de procesamiento: {processing_time:.2f} segundos.")

# 2. Guardar en archivo .txt (incluyendo el tiempo)
print("Guardando resumen de conteos...")
summary_file_path = "Total_clases_Tesseract2.txt" # El nombre de tu archivo de resumen
try:
    with open(summary_file_path, "w", encoding='utf-8') as f:
        f.write("--- Resumen de Conteo Único (cuente el total de cada clase) ---\n")
        f.write(f"Total de coches únicos rastreados: {total_cars}\n")
        f.write(f"Total de personas únicas rastreadas: {total_people}\n")
        f.write(f"Total de motos únicas rastreadas: {total_motos}\n")
        f.write(f"Total de matrículas únicas rastreadas: {total_plates}\n")
        f.write("\n") # Añade una línea en blanco
        f.write(f"Tiempo total de procesamiento: {processing_time:.2f} segundos.\n") # Añade el tiempo al archivo
        
    print(f"Resumen de conteos guardado en '{summary_file_path}'") # Corregido el nombre del archivo
except Exception as e:
    print(f"Error al guardar el resumen: {e}")

Configurazione del progetto...
Caricamento modelli...
Tesseract trovato e configurato.
Scaldando il modello YOLO...
Modello YOLO caricato e pronto.
Definizione funzioni di supporto...
Inizio elaborazione video...
  Elaborato frame 100...
  Elaborato frame 200...
  Elaborato frame 300...

Elaborazione video completata.
Tempo totale di elaborazione: 309.59 secondi.
Guardando archivo CSV...
¡Guardado completado! Video guardado en 'risultato_Tesseract2.mp4' y CSV en 'risultati_Tesseract2.csv'.

--- Resumen de Conteo Único (cuente el total de cada clase) ---
Total de coches únicos rastreados: 5
Total de personas únicas rastreadas: 6
Total de motos únicas rastreadas: 0
Total de matrículas únicas rastreadas: 6

Tiempo total de procesamiento: 309.59 segundos.
Guardando resumen de conteos...
Resumen de conteos guardado en 'Total_clases_Tesseract2.txt'


In [ ]:
#EASYOCR

import os
import cv2
import pandas as pd
import easyocr      
from ultralytics import YOLO
import numpy as np
import time

# -----------------------------------------------------------------
# Celda 1: Configuración
# -----------------------------------------------------------------
print("Configurando el proyecto...")

# --- CONFIGURACIÓN PRINCIPAL ---

# Ruta al modelo YOLO entrenado
MODEL_PATH = r"C:\Users\ricca\OneDrive - Alma Mater Studiorum Università di Bologna\Desktop\Vision Por computador\lab\4\yolo\yolo\my_model\train\weights\best.pt"

# Archivo de video a analizar
#VIDEO_SOURCE_PATH = r"C:\Users\ricca\OneDrive - Alma Mater Studiorum Università di Bologna\Desktop\Vision Por computador\lab\4\yolo\yolo\videoProva.mp4"
VIDEO_SOURCE_PATH = r"C:\Users\ricca\OneDrive - Alma Mater Studiorum Università di Bologna\Desktop\Vision Por computador\lab\4\video\videoTelefono4.mp4"

# Nombre del archivo de video de salida
OUTPUT_VIDEO_PATH = "risultato_EASYOCR2.mp4"

# Nombre del archivo CSV de salida (formato requerido)
OUTPUT_CSV_PATH = "risultati_EASYOCR2.csv"

# Umbral de confianza mínimo
CONF_THRESHOLD = 0.4 

# -----------------------------------------------------------------
# Celda 2: Importaciones y Carga de Modelos
# -----------------------------------------------------------------
print("Cargando modelos...")

# Carga el modelo YOLO
try:
    model = YOLO(MODEL_PATH)
    print("Calentando el modelo YOLO...")
    dummy_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
    model.track(dummy_frame, persist=True, verbose=False)
    print("Modelo YOLO cargado y listo.")
except Exception as e:
    print(f"Error al cargar el modelo YOLO: {e}")
    # raise e 

# Cargar el lector EasyOCR
try:
    print("Cargando modelo EasyOCR...")
    ocr_reader = easyocr.Reader(['en'], gpu=True) 
    print("EasyOCR cargado en GPU.")
except Exception as e:
    print(f"Error al cargar EasyOCR en GPU: {e}.")
    print("Intentando cargar EasyOCR en CPU (más lento)...")
    try:
        ocr_reader = easyocr.Reader(['en'], gpu=False)
        print("EasyOCR cargado en CPU.")
    except Exception as e_cpu:
        print(f"Error crítico: No se pudo cargar EasyOCR: {e_cpu}")
        # raise e_cpu

# -----------------------------------------------------------------
# Celda 3: Funciones de Soporte (OCR y Preprocesamiento)
# -----------------------------------------------------------------
print("Definiendo funciones de soporte...")

def canonical_classname(name: str) -> str:
    """Normaliza los nombres de las clases."""
    s = name.strip().lower().replace('_', ' ').replace('-', ' ')
    plate_aliases = {'license plate','licence plate','number plate','plate','vehicle plate','car plate','licence','license', 'matricula'}
    moto_aliases = {'motorcycle','motorbike','moped','scooter', 'moto'}
    car_aliases = {'car','automobile','vehicle','auto', 'coche'}
    person_aliases = {'person','pedestrian','people', 'persona'}
    if s in plate_aliases: return 'license-plate'
    if s in moto_aliases: return 'motorcycle'
    if s in car_aliases: return 'car'
    if s in person_aliases: return 'person'
    return s

def get_box_center(box):
    """Calcula el centro de un bounding box."""
    x1, y1, x2, y2 = box
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

# MODIFICACIÓN: Función OCR para EasyOCR (ahora devuelve solo el texto)
def run_ocr_on_plate(plate_image):
    """
    Ejecuta EasyOCR en una imagen de matrícula.
    Esta función ahora devuelve solo el texto limpio, nada más.
    """
    if plate_image.size == 0:
        return ""

    text = ""
    try:
        # EasyOCR prefiere la imagen BGR original.
        result = ocr_reader.readtext(plate_image, detail=1, paragraph=False)

        if result:
            text = result[0][1]
            conf = result[0][2] # Tomamos la confianza de EasyOCR
            
            # Limpiar el texto
            text = "".join(c for c in text if c.isalnum()).upper()

            # Filtro: Aceptar solo si tiene una confianza decente Y una longitud plausible
            # Bajamos el umbral para permitir que el sistema de votación funcione
            if conf > 0.1 and len(text) >= 3: 
                return text # Devolver solo el texto limpio
                
    except Exception as e:
        pass
    
    return "" # Devolver vacío en todos los demás casos

# -----------------------------------------------------------------
# Celda 4: Procesamiento Principal
# -----------------------------------------------------------------
print("Iniciando procesamiento de video...")

csv_results = []
start_time = time.time()

# --- MODIFICACIÓN: Diccionarios para la "Memoria" OCR ---
# Guarda el historial de lecturas para cada track_id
ocr_history = {} 

# Set para rastrear los IDs únicos para el conteo total
unique_object_ids = {
    'car': set(), 'person': set(), 'license-plate': set(), 'motorcycle': set()
}

cap = cv2.VideoCapture(VIDEO_SOURCE_PATH)
if not cap.isOpened():
    print(f"ERROR: No se pudo abrir el video fuente: {VIDEO_SOURCE_PATH}")
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    recorder = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("\nProcesamiento de video completado.")
            break

        frame_number += 1
        
        # Ejecutar seguimiento con YOLO
        results = model.track(frame, persist=True, conf=CONF_THRESHOLD, verbose=False)

        # Set para rastrear los IDs activos en este frame
        current_frame_active_ids = set()

        if results[0].boxes and len(results[0].boxes) > 0:
            boxes = results[0].boxes.cpu().numpy()
            class_names = results[0].names
            
            cars_in_frame = []
            plates_in_frame = []
            people_in_frame = []
            motos_in_frame = []

            has_tracking_ids = boxes.id is not None

            for i in range(len(boxes)):
                class_id = int(boxes.cls[i])
                class_name = canonical_classname(class_names[class_id])
                track_id = int(boxes.id[i]) if has_tracking_ids else -1
                conf = float(boxes.conf[i])
                bbox = boxes.xyxy[i].astype(int)
                
                obj_data = {'bbox': bbox, 'track_id': track_id, 'conf': conf, 'class_name': class_name}

                if track_id != -1:
                    # Añadir el ID al set para el conteo único
                    if class_name in unique_object_ids:
                        unique_object_ids[class_name].add(track_id)
                    # Añadir a los IDs activos para la limpieza de memoria OCR
                    if class_name == 'license-plate':
                        current_frame_active_ids.add(track_id)

                if class_name == 'car':
                    cars_in_frame.append(obj_data)
                elif class_name == 'license-plate':
                    plates_in_frame.append(obj_data)
                elif class_name == 'person':
                    people_in_frame.append(obj_data)
                elif class_name == 'motorcycle':
                    motos_in_frame.append(obj_data)

            # Lógica OCR  
            for plate in plates_in_frame:
                track_id = plate['track_id']
                if track_id == -1: # Saltar si el tracker aún no ha asignado un ID
                    continue

                x1, y1, x2, y2 = plate['bbox']
                plate_img = frame[y1:y2, x1:x2]
                
                # Ejecutar OCR (se ejecuta en cada frame para obtener más "votos")
                text = run_ocr_on_plate(plate_img) # Usa la función EasyOCR
                
                # Si EasyOCR ha leído algo, actualizar la memoria
                if text:
                    if track_id not in ocr_history:
                        ocr_history[track_id] = {} # Inicializar memoria para este ID
                    
                    # Añadir un "voto" para este texto
                    ocr_history[track_id][text] = ocr_history[track_id].get(text, 0) + 1
                
                # Ahora, determinar el texto ESTABLE y su confianza
                if track_id in ocr_history:
                    history_for_id = ocr_history[track_id]
                    
                    # Encontrar el texto con más votos
                    best_text = max(history_for_id, key=history_for_id.get)
                    
                    # Calcular la "confianza de estabilidad"
                    total_votes = sum(history_for_id.values())
                    best_votes = history_for_id[best_text]
                    stability_conf = best_votes / total_votes
                    
                    # Guardar el resultado estable
                    plate['text'] = best_text
                    plate['text_conf'] = stability_conf
                
                # Dibujar el recuadro de la matrícula
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                
                # Dibujar el texto ESTABLE y su confianza
                if plate.get('text', ''): 
                    text_to_draw = f"{plate['text']} ({plate.get('text_conf', 0):.2f})"
                    cv2.putText(frame, text_to_draw, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # --- Asociar matrículas a los coches y rellenar el CSV ---
            for car in cars_in_frame:
                car_bbox = car['bbox']
                associated_plate = None
                for plate in plates_in_frame:
                    center_x, center_y = get_box_center(plate['bbox'])
                    if car_bbox[0] < center_x < car_bbox[2] and car_bbox[1] < center_y < car_bbox[3]:
                        associated_plate = plate
                        break
                
                row = {
                    'fotograma': frame_number, 'tipo_oggetto': 'car', 'confianza': f"{car['conf']:.2f}",
                    'identificador_tracking': f"car_{car['track_id']}", 'x1': car_bbox[0], 'y1': car_bbox[1], 'x2': car_bbox[2], 'y2': car_bbox[3],
                    'matrícula_en_su_caso': 'no', 'confianza_matricula': 0, 'mx1': 0, 'my1': 0, 'mx2': 0, 'my2': 0, 'texto_matricula': ''
                }

                if associated_plate:
                    plate_bbox = associated_plate['bbox']
                    row.update({
                        'matrícula_en_su_caso': 'si',
                        'confianza_matricula': f"{associated_plate.get('text_conf', 0):.2f}", # Usa la confianza de estabilidad
                        'mx1': plate_bbox[0], 'my1': plate_bbox[1], 'mx2': plate_bbox[2], 'my2': plate_bbox[3],
                        'texto_matricula': associated_plate.get('text', '') # Usa el texto estable
                    })
                
                csv_results.append(row)
                
                cv2.rectangle(frame, (car_bbox[0], car_bbox[1]), (car_bbox[2], car_bbox[3]), (0, 255, 0), 2)
                cv2.putText(frame, f"Car ID:{car['track_id']}", (car_bbox[0], car_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Añadir las personas al CSV y dibujarlas
            for person in people_in_frame:
                p_bbox = person['bbox']
                csv_results.append({
                    'fotograma': frame_number, 'tipo_oggetto': 'person', 'confianza': f"{person['conf']:.2f}",
                    'identificador_tracking': f"person_{person['track_id']}", 'x1': p_bbox[0], 'y1': p_bbox[1], 'x2': p_bbox[2], 'y2': p_bbox[3],
                    'matrícula_en_su_caso': 'no', 'confianza_matricula': 0, 'mx1': 0, 'my1': 0, 'mx2': 0, 'my2': 0, 'texto_matricula': ''
                })
                cv2.rectangle(frame, (p_bbox[0], p_bbox[1]), (p_bbox[2], p_bbox[3]), (255, 0, 0), 2)
                cv2.putText(frame, f"Person ID:{person['track_id']}", (p_bbox[0], p_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Añadir las motos al CSV y dibujarlas
            for moto in motos_in_frame:
                m_bbox = moto['bbox']
                csv_results.append({
                    'fotograma': frame_number, 'tipo_oggetto': 'motorcycle', 'confianza': f"{moto['conf']:.2f}",
                    'identificador_tracking': f"motorcycle_{moto['track_id']}", 'x1': m_bbox[0], 'y1': m_bbox[1], 'x2': m_bbox[2], 'y2': m_bbox[3],
                    'matrícula_en_su_caso': 'no', 'confianza_matricula': 0, 'mx1': 0, 'my1': 0, 'mx2': 0, 'my2': 0, 'texto_matricula': ''
                })
                cv2.rectangle(frame, (m_bbox[0], m_bbox[1]), (m_bbox[2], m_bbox[3]), (0, 165, 255), 2) # Color naranja
                cv2.putText(frame, f"Moto ID:{moto['track_id']}", (m_bbox[0], m_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        recorder.write(frame)
        
        # Imprimir actualización cada 100 frames
        if frame_number % 100 == 0:
            print(f"  Procesado frame {frame_number}...")

        # Mostrar vista previa
        cv2.imshow("Anteprima - EasyOCR ESTABLE", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        # --- MODIFICACIÓN: Limpieza de la memoria OCR ---
        # Eliminar IDs que ya no están activos
        for old_id in list(ocr_history.keys()):
            if old_id not in current_frame_active_ids:
                del ocr_history[old_id]

    # Liberar recursos
    cap.release()
    recorder.release()
    cv2.destroyAllWindows()
    for i in range(5): cv2.waitKey(1) # Forzar el cierre de las ventanas
    
    end_time = time.time()
    print(f"Tiempo total de procesamiento: {end_time - start_time:.2f} segundos.")

# -----------------------------------------------------------------
# Celda 5: Guardado de CSV y Conteo Total
# -----------------------------------------------------------------
print("Guardando archivo CSV...")

if csv_results:
    df = pd.DataFrame(csv_results)
    
    # Asegurar el orden correcto de las columnas según lo requerido
    colonne_richieste = [
        'fotograma', 'tipo_oggetto', 'confianza', 'identificador_tracking',
        'x1', 'y1', 'x2', 'y2', 'matrícula_en_su_caso', 'confianza_matricula',
        'mx1', 'my1', 'mx2', 'my2', 'texto_matricula'
    ]
    
    # Rellenar columnas faltantes si no existen (seguridad)
    for col in colonne_richieste:
        if col not in df.columns:
            # Crear una columna vacía con el tipo de dato correcto
            if 'x' in col or 'y' in col or 'fotograma' in col:
                df[col] = 0
            elif 'texto' in col:
                df[col] = ''
            elif 'confianza' in col:
                df[col] = 0.0
            else:
                df[col] = 'no'
            
    df = df[colonne_richieste]
    
    df.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"¡Guardado completado! Video guardado en '{OUTPUT_VIDEO_PATH}' y CSV en '{OUTPUT_CSV_PATH}'.")
else:
    print("No se detectaron objetos durante el procesamiento. No se guardó ningún archivo CSV.")

# Imprimir y guardar el conteo total y el tiempo
print("\n--- Resumen de Conteo Único (cuente el total de cada clase) ---")

# Calcular totales
total_cars = len(unique_object_ids.get('car', set()))
total_people = len(unique_object_ids.get('person', set()))
total_plates = len(unique_object_ids.get('license-plate', set()))
total_motos = len(unique_object_ids.get('motorcycle', set()))

# Calcular tiempo (usando las variables 'end_time' y 'start_time' de la Celda 4)
try:
    processing_time = end_time - start_time
except NameError:
    print("Advertencia: No se encontraron las variables start_time/end_time. El tiempo no se guardará.")
    processing_time = -1.0 # Valor por defecto

# 1. Imprimir en consola (incluyendo el tiempo)
print(f"Total de coches únicos rastreados: {total_cars}")
print(f"Total de personas únicas rastreadas: {total_people}")
print(f"Total de motos únicas rastreadas: {total_motos}")
print(f"Total de matrículas únicas rastreadas: {total_plates}")
print(f"\nTiempo total de procesamiento: {processing_time:.2f} segundos.")

# 2. Guardar en archivo .txt (incluyendo el tiempo)
print("Guardando resumen de conteos...")
summary_file_path = "Total_clases_EasyOCR2.txt" # El nombre de tu archivo de resumen
try:
    with open(summary_file_path, "w", encoding='utf-8') as f:
        f.write("--- Resumen de Conteo Único (cuente el total de cada clase) ---\n")
        f.write(f"Total de coches únicos rastreados: {total_cars}\n")
        f.write(f"Total de personas únicas rastreadas: {total_people}\n")
        f.write(f"Total de motos únicas rastreadas: {total_motos}\n")
        f.write(f"Total de matrículas únicas rastreadas: {total_plates}\n")
        f.write("\n") # Añade una línea en blanco
        f.write(f"Tiempo total de procesamiento: {processing_time:.2f} segundos.\n") # Añade el tiempo al archivo
        
    print(f"Resumen de conteos guardado en '{summary_file_path}'")
except Exception as e:
    print(f"Error al guardar el resumen: {e}")

Configurando el proyecto...
Cargando modelos...
Calentando el modelo YOLO...


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Modelo YOLO cargado y listo.
Cargando modelo EasyOCR...
EasyOCR cargado en GPU.
Definiendo funciones de soporte...
Iniciando procesamiento de video...


c:\Users\ricca\anaconda3\envs\VC_P4\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  Procesado frame 100...
  Procesado frame 200...
  Procesado frame 300...

Procesamiento de video completado.
Tiempo total de procesamiento: 210.77 segundos.
Guardando archivo CSV...
¡Guardado completado! Video guardado en 'risultato_EASYOCR2.mp4' y CSV en 'risultati_EASYOCR2.csv'.

--- Resumen de Conteo Único (cuente el total de cada clase) ---
Total de coches únicos rastreados: 5
Total de personas únicas rastreadas: 6
Total de motos únicas rastreadas: 0
Total de matrículas únicas rastreadas: 6

Tiempo total de procesamiento: 210.77 segundos.
Guardando resumen de conteos...
Resumen de conteos guardado en 'Total_clases_EasyOCR2.txt'
